In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy
import seaborn as sns
import psycopg2

sns.set(style="white", color_codes=True)
%matplotlib inline

In [2]:
#Создаем подключение и смотри объем данных
conn = psycopg2.connect(host='rc1c-fhrb9f1e0l9g611h.mdb.yandexcloud.net',
                        port=6432,
                        dbname='hr-analytics',
                        sslmode='require',
                        user='analytics',
                        password='HRanalytics')
cur = conn.cursor()

cur.execute("""SELECT COUNT(*) FROM orders""")
query_results = cur.fetchall()
print(query_results)
cur.execute("""SELECT COUNT(*) FROM delivery_windows""")
query_results = cur.fetchall()
print(query_results)
cur.execute("""SELECT COUNT(*) FROM stores""")
query_results = cur.fetchall()
print(query_results)
cur.execute("""SELECT COUNT(*) FROM addresses""")
query_results = cur.fetchall()
print(query_results)
cur.execute("""SELECT COUNT(*) FROM replacements""")
query_results = cur.fetchall()
print(query_results)
cur.execute("""SELECT COUNT(*) FROM cancellations""")
query_results = cur.fetchall()
print(query_results)

[(720433,)]
[(204343,)]
[(162,)]
[(1492303,)]
[(1250244,)]
[(1441605,)]


In [3]:
#Загружаем данные
orders = pd.read_sql("""SELECT * FROM orders""", conn)
addresses = pd.read_sql("""SELECT * FROM addresses""", conn)
delivery_windows = pd.read_sql("""SELECT * FROM delivery_windows""", conn)
stores = pd.read_sql("""SELECT * FROM stores""", conn)
replacements = pd.read_sql("""SELECT * FROM replacements""", conn)
cancellations = pd.read_sql("""SELECT * FROM cancellations""", conn)

In [4]:
orders.sample(5)

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,state,shipped_at,created_at,ship_address_id,store_id,delivery_window_id
593679,1687250,365618,158.0,238.0,3265,0.0,19322,38,shipped,2019-07-01 15:23:38,2019-07-01 10:45:19,2299215,91,579727
102898,1105694,239782,0.0,299.0,3306,0.0,8000,4,shipped,2019-02-07 18:22:00,2019-02-02 04:35:34,1467723,3,392518
7761,1075375,167400,244.0,244.0,5192,0.0,31389,47,shipped,2019-01-23 12:11:00,2019-01-21 16:26:59,1167338,1,376096
406043,2182920,821998,0.0,299.0,3747,0.0,6935,23,shipped,2019-09-03 13:57:35,2019-09-01 18:42:37,3100227,21,692462
638679,4287415,1278470,158.0,158.0,6743,0.0,21121,38,shipped,2019-11-21 08:24:22,2019-11-20 10:28:58,5667473,98,879306


In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720433 entries, 0 to 720432
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  720433 non-null  int64         
 1   user_id             720433 non-null  int64         
 2   total_cost          720433 non-null  float64       
 3   cost                720433 non-null  float64       
 4   item_total          720433 non-null  int64         
 5   promo_total         720433 non-null  float64       
 6   total_weight        720433 non-null  int64         
 7   total_quantity      720433 non-null  int64         
 8   state               720433 non-null  object        
 9   shipped_at          720433 non-null  datetime64[ns]
 10  created_at          720433 non-null  datetime64[ns]
 11  ship_address_id     720433 non-null  int64         
 12  store_id            720433 non-null  int64         
 13  delivery_window_id  720433 no

In [6]:
orders.describe()

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,ship_address_id,store_id,delivery_window_id
count,7.204330e+05,7.204330e+05,720433.000000,720433.000000,720433.000000,720433.000000,7.204330e+05,720433.000000,7.204330e+05,720433.000000,7.204330e+05
mean,2.747200e+06,6.385489e+05,171.820556,252.459416,4936.180057,-51.434371,2.458678e+04,42.923062,3.477356e+06,50.902284,6.905123e+05
std,1.446098e+06,5.405112e+05,152.639804,144.192244,4801.748586,234.262256,3.312988e+04,79.485753,1.832337e+06,56.695703,2.156267e+05
min,7.569140e+05,3.100000e+01,-148.000000,0.000000,0.000000,-39899.300000,0.000000e+00,0.000000,6.017000e+03,1.000000,2.264580e+05
25%,1.496435e+06,1.867060e+05,98.000000,199.000000,2673.000000,0.000000,1.043100e+04,20.000000,1.861411e+06,9.000000,5.237300e+05
50%,2.423437e+06,3.630850e+05,158.000000,249.000000,3964.000000,0.000000,1.800000e+04,32.000000,3.190749e+06,14.000000,7.361470e+05
75%,4.129816e+06,1.129740e+06,263.000000,299.000000,5918.000000,0.000000,2.990000e+04,50.000000,5.123160e+06,90.000000,8.803310e+05
max,5.444849e+06,1.837484e+06,30000.000000,30000.000000,481195.000000,0.000000,7.005600e+06,36040.000000,7.146722e+06,317.000000,1.242161e+06


In [7]:
#Проверим наличие заказов с нулевой корзиной
orders.query('item_total == 0')

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,state,shipped_at,created_at,ship_address_id,store_id,delivery_window_id
25087,2457003,777240,0.0,0.0,0,0.0,500,4,shipped,2019-09-24 13:33:58,2019-09-24 10:10:43,3520882,1,732756
81574,2828415,1112015,0.0,299.0,0,0.0,4625,13,shipped,2019-10-20 10:04:05,2019-10-18 20:58:03,4082440,2,796140
108509,1631533,416183,0.0,299.0,0,0.0,18000,18,shipped,2019-06-21 14:17:06,2019-06-21 09:52:24,2213321,3,561538
320701,5126949,1758530,98.0,249.0,0,-199.0,10950,11,shipped,2019-12-11 05:46:06,2019-12-10 20:02:23,6751812,12,943952
326495,881444,173854,0.0,299.0,0,0.0,705,3,shipped,2018-10-30 05:50:49,2018-10-29 16:01:04,1104838,14,282229
444602,2094365,674612,0.0,0.0,0,0.0,3385,15,shipped,2019-08-25 17:24:13,2019-08-25 13:26:08,2938463,70,675904
504839,3838920,1410293,199.0,199.0,0,0.0,5810,7,shipped,2019-11-11 09:12:57,2019-11-09 07:23:24,5203891,63,852220
701620,1528533,353833,99.0,149.0,0,0.0,3723,16,shipped,2019-06-01 04:41:19,2019-05-30 15:04:09,1772737,108,530572
716938,5004403,1727727,99.0,199.0,0,0.0,11771,18,shipped,2019-12-08 07:51:02,2019-12-08 03:53:15,6378487,169,932160


In [47]:
#Записываем в файл
orders.query('item_total == 0').to_excel("output.xlsx", sheet_name='ZeroTotal') 

In [9]:
#Проверим наличие заказов у которых финальная стоимость доставки больше начальной
orders.query('total_cost > cost')

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,state,shipped_at,created_at,ship_address_id,store_id,delivery_window_id


In [10]:
#Проверим наличие заказов, которые приехали до создания
orders.query('shipped_at < created_at')

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,state,shipped_at,created_at,ship_address_id,store_id,delivery_window_id
380008,980548,202199,0.0,199.0,2902,-500.0,3325,11,shipped,2018-12-08 10:27:52,2018-12-11 10:24:44,1275667,18,330168
380009,980564,202201,0.0,199.0,2016,-500.0,2850,8,shipped,2018-12-08 10:32:40,2018-12-11 10:30:02,1275696,18,330167
380010,980590,202207,0.0,199.0,2237,-500.0,6600,12,shipped,2018-12-08 10:41:15,2018-12-11 10:38:50,1275724,18,330167


In [48]:
#Записываем в файл
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    orders.query('shipped_at < created_at').to_excel(writer, sheet_name='ArrBefCreate') 

In [12]:
#Поссмотрим какие состояния имеются в таблице
orders.state.value_counts()

shipped     720404
canceled        15
pending         12
shipping         1
ready            1
Name: state, dtype: int64

In [13]:
#Подробно рассмотрим заказы с состояниями: pending, shipping, ready
orders[(orders.state == 'pending') | (orders.state == 'shipping') | (orders.state == 'ready')]

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,state,shipped_at,created_at,ship_address_id,store_id,delivery_window_id
130587,782142,146383,0.0,0.0,43471,-8740.09,50111,178,pending,2018-09-15 23:56:00,2018-09-08 17:28:36,931411,8,243269
171645,4885858,929117,329.0,329.0,2594,0.00,35220,45,pending,2019-12-09 15:11:21,2019-12-05 10:53:07,5878011,8,934773
171660,4889144,1687572,258.0,309.0,14266,-250.00,54096,95,shipping,2019-12-07 15:31:31,2019-12-05 11:58:13,6397921,8,926639
207748,1282717,292509,0.0,244.0,15757,-893.00,38555,106,pending,2019-04-13 10:33:00,2019-04-12 09:56:29,1741011,10,465691
225773,2972260,1154572,324.0,324.0,7078,0.00,34751,63,pending,2019-10-28 18:08:10,2019-10-22 12:24:56,4250505,10,815852
265656,3206997,1224575,299.0,299.0,6500,0.00,22864,77,pending,2019-11-03 20:16:02,2019-10-30 17:52:10,4513520,11,831494
315284,4033720,489016,299.0,299.0,642,0.00,4880,11,pending,2019-11-14 11:53:59,2019-11-13 17:56:33,5372270,12,860131
316130,4178073,549019,299.0,299.0,4991,0.00,9595,20,pending,2019-11-21 14:38:54,2019-11-17 14:48:47,4311159,12,878182
329109,970114,153851,299.0,299.0,3588,0.00,17183,32,pending,2018-12-06 15:58:00,2018-12-05 23:00:05,1240279,14,324458
377796,5199839,187568,299.0,299.0,5055,0.00,24558,63,pending,2019-12-12 19:44:11,2019-12-12 15:02:08,6842569,14,947413


In [49]:
#Записываем в файл
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    orders[(orders.state == 'pending') | (orders.state == 'shipping') | (orders.state == 'ready')].to_excel(writer, 
    sheet_name='StateErr')

In [15]:
#Подробно рассмотрим отмененные заказы
orders[orders.state == "canceled"]

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,state,shipped_at,created_at,ship_address_id,store_id,delivery_window_id
119159,2917814,1129004,299.0,299.0,725,0.0,11655,25,canceled,2019-10-25 11:14:15,2019-10-21 15:39:32,4193170,3,808418
130068,765089,123890,199.0,199.0,15935,0.0,22650,21,canceled,2018-08-30 12:53:16,2018-08-30 08:54:24,868201,8,230002
193825,4264275,961184,299.0,299.0,6172,0.0,24206,49,canceled,2019-11-20 11:51:41,2019-11-19 17:42:08,4941212,9,875426
266477,3408961,1232248,299.0,299.0,3613,0.0,23870,40,canceled,2019-11-06 14:51:55,2019-11-03 18:54:01,4757922,11,839017
324253,820764,156619,0.0,0.0,851,0.0,240,6,canceled,2018-09-28 15:12:57,2018-09-28 15:08:01,1002269,14,254206
327847,924962,186248,0.0,299.0,2474,0.0,2200,3,canceled,2018-11-16 11:43:38,2018-11-16 11:31:48,1181945,14,303088
368836,4233456,773943,566.0,566.0,13908,0.0,142180,126,canceled,2019-11-25 07:55:17,2019-11-19 04:38:56,5521222,14,888803
369062,4262575,1087110,359.0,359.0,8870,0.0,41507,69,canceled,2019-11-20 13:38:16,2019-11-19 16:58:25,5639697,14,875630
380031,1018002,123143,149.0,149.0,6906,0.0,20000,20,canceled,2018-12-28 15:34:58,2018-12-28 15:34:47,1332920,18,348384
380032,1018024,123143,149.0,149.0,6993,0.0,20000,20,canceled,2018-12-28 15:41:13,2018-12-28 15:40:59,1332945,18,348383


In [16]:
orders.query("-promo_total/(item_total+total_cost) >= 0.8")

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,state,shipped_at,created_at,ship_address_id,store_id,delivery_window_id
230,765582,142070,0.0,0.0,3138,-3000.0,26175,34,shipped,2018-08-31 10:02:00,2018-08-30 12:18:27,904196,1,230768
261,766601,142290,0.0,0.0,92,-500.0,10930,15,shipped,2018-09-03 21:14:22,2018-08-30 19:40:29,905771,1,233265
465,774026,144172,0.0,0.0,16,-100.0,2016,2,shipped,2018-09-05 19:22:20,2018-09-04 08:36:45,917813,1,234916
492,775137,144466,0.0,0.0,385,-500.0,7370,20,shipped,2018-09-05 20:08:15,2018-09-04 20:10:48,919426,1,234919
593,778466,145349,0.0,0.0,3429,-3000.0,6447,23,shipped,2018-09-07 17:41:23,2018-09-06 18:59:09,924876,1,236571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715942,4213957,1502591,99.0,199.0,1080,-1000.0,10120,16,shipped,2019-11-19 16:30:01,2019-11-18 14:12:04,5558092,169,874950
715956,4232949,1195830,99.0,199.0,716,-1000.0,17600,34,shipped,2019-11-20 13:51:01,2019-11-19 04:03:36,4387093,169,877711
717321,5327853,1820090,99.0,199.0,52,-300.0,9400,15,shipped,2019-12-17 04:35:41,2019-12-15 18:09:43,6156667,169,968318
717351,5351725,1822867,99.0,199.0,236,-300.0,2270,16,shipped,2019-12-17 11:21:32,2019-12-16 11:24:39,7027979,169,968323


In [50]:
#Записываем в файл
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    orders.query("-promo_total/(item_total+total_cost) >= 0.8").to_excel(writer,
        sheet_name='BigPromo')

In [18]:
#Посмотрим количество заказов по магазинам и количество самих магазинов
print("Количество магазинов в заказах: {}".format(orders.store_id.value_counts().count()))
orders.store_id.value_counts()

Количество магазинов в заказах: 71


14     57606
2      52511
12     47574
8      45599
1      44557
       ...  
259      228
248      221
254      193
317        3
179        1
Name: store_id, Length: 71, dtype: int64

In [19]:
#Посмотрим количество магазинов согласно таблице stores
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   id           162 non-null    int64
 1   city         162 non-null    int64
 2   retailer_id  162 non-null    int64
dtypes: int64(3)
memory usage: 3.9 KB


In [20]:
#Посмотрим заказы из магазинов 317 и 179
orders.query('store_id==317 | store_id==179')

,id,user_id,total_cost,cost,item_total,promo_total,total_weight,total_quantity,state,shipped_at,created_at,ship_address_id,store_id,delivery_window_id
465234,4577698,178734,199.0,199.0,903,0.0,1000,1,shipped,2019-11-27 17:00:54,2019-11-27 16:54:12,6043941,179,897760
478858,5426721,1834593,108.0,259.0,5396,-250.0,28991,49,shipped,2019-12-18 18:26:40,2019-12-18 10:33:30,7125140,317,996053
478859,5432426,1833706,98.0,249.0,4343,0.0,13295,15,shipped,2019-12-18 15:47:14,2019-12-18 12:46:44,7130242,317,996052
478860,5439136,684383,128.0,279.0,3048,0.0,32973,53,shipped,2019-12-18 18:47:43,2019-12-18 15:34:14,3119768,317,996054


In [21]:
#Создадим таблицу, показывающую количество отмен и замен в заказе
a = cancellations.order_id.value_counts()
b = replacements.order_id.value_counts()
ba = pd.DataFrame({'id':b.index, 'replacements':b.values})
ab = pd.DataFrame({'id':a.index, 'cancellations':a.values})
a = ab.merge(ba, how='outer').fillna(0).convert_dtypes()
ordStat = orders.query('state == "shipped"')[['id', 'store_id', 'total_quantity']].merge(a, 
    how='left').fillna(0).convert_dtypes()
del a, b, ba, ab
ordStat

,id,store_id,total_quantity,cancellations,replacements
0,756927,1,39,8,0
1,756963,1,71,9,0
2,756983,1,32,4,0
3,756985,1,13,1,3
4,757066,1,24,2,4
...,...,...,...,...,...
720399,5423310,235,71,1,4
720400,5423488,235,13,0,3
720401,5424466,235,7,0,0
720402,5426016,235,34,1,3


In [22]:
#Проверим наличие привезенных заказов, у которых отменили все товары
ordStat[ordStat.total_quantity == ordStat.cancellations]

,id,store_id,total_quantity,cancellations,replacements
128,761102,1,10,10,0
410,772241,1,1,1,0
1214,802319,1,1,1,1
1278,804571,1,1,1,0
2132,834828,1,1,1,0
...,...,...,...,...,...
710199,4614121,108,1,1,0
714783,3052697,169,2,2,0
714903,3192070,169,11,11,0
717058,5113384,169,1,1,0


In [51]:
#Записываем в файл
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    ordStat[ordStat.total_quantity == ordStat.cancellations].to_excel(writer, 
        sheet_name='WhyNotCancelled')

In [24]:
#Составим таблицу, показывающую количество отмен и замен в магазинах
storeStat = stores[['id', 'retailer_id']]
storeStat.index = storeStat.id
storeStat.index.name = "id"
storeStat['total_quantity'] = storeStat['id']
storeStat['cancellations'] = storeStat['id']
storeStat['replacements'] = storeStat['id']
del storeStat['id']
for storeids in storeStat.index:
    storeStat.loc[storeids]['total_quantity'] = ordStat[ordStat.store_id == storeids].total_quantity.sum()
    storeStat.loc[storeids]['cancellations'] = ordStat[ordStat.store_id == storeids].cancellations.sum()
    storeStat.loc[storeids]['replacements'] = ordStat[ordStat.store_id == storeids].replacements.sum()
storeStat['replace_perc'] = storeStat['replacements']/storeStat['total_quantity']
storeStat['cancell_perc'] = storeStat['cancellations']/storeStat['total_quantity']
storeStat = storeStat.fillna(0).convert_dtypes()
storeStat

,retailer_id,total_quantity,cancellations,replacements,replace_perc,cancell_perc
id,,,,,,
1,1,2216879,82717,76086,0.034321,0.037312
2,1,2548808,83923,68769,0.026981,0.032926
3,1,1404494,65711,44966,0.032016,0.046786
8,1,2103569,92698,79204,0.037652,0.044067
9,1,1062630,45637,44582,0.041954,0.042947
...,...,...,...,...,...,...
1174,59,0,0,0,0.0,0.0
1175,59,0,0,0,0.0,0.0
1205,15,0,0,0,0.0,0.0


In [25]:
storeStat.describe()

,retailer_id,total_quantity,cancellations,replacements,replace_perc,cancell_perc
count,162.000000,1.620000e+02,162.000000,162.000000,162.000000,162.000000
mean,15.419753,1.908736e+05,7973.765432,7198.537037,0.026714,0.022944
std,15.043923,5.184279e+05,19633.693633,17005.394688,0.035145,0.028257
min,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,8.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,15.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,16.000000,7.955800e+04,4115.750000,6359.000000,0.057232,0.049251
max,61.000000,2.741404e+06,101285.000000,92577.000000,0.114596,0.092540


In [26]:
#Посмотрим наличие магазинов с нулем заказов
storeStat[storeStat.total_quantity == 0]

,retailer_id,total_quantity,cancellations,replacements,replace_perc,cancell_perc
id,,,,,,
178,8,0,0,0,0.0,0.0
263,8,0,0,0,0.0,0.0
267,15,0,0,0,0.0,0.0
268,15,0,0,0,0.0,0.0
275,15,0,0,0,0.0,0.0
...,...,...,...,...,...,...
1174,59,0,0,0,0.0,0.0
1175,59,0,0,0,0.0,0.0
1205,15,0,0,0,0.0,0.0


In [27]:
storeStat[storeStat.total_quantity > 0].describe()

,retailer_id,total_quantity,cancellations,replacements,replace_perc,cancell_perc
count,71.000000,7.100000e+01,71.000000,71.000000,71.000000,71.000000
mean,8.633803,4.355143e+05,18193.661972,16424.830986,0.060953,0.052351
std,6.167286,7.142120e+05,26419.970444,22614.525224,0.026910,0.016583
min,1.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.810950e+04,1078.000000,1379.500000,0.035039,0.043119
50%,8.000000,1.026650e+05,6582.000000,7835.000000,0.064738,0.051240
75%,15.000000,4.735195e+05,20791.500000,19003.500000,0.079491,0.060787
max,16.000000,2.741404e+06,101285.000000,92577.000000,0.114596,0.092540


In [28]:
#Посмотрим наличие магазинов с процентом замены больше 10
storeStat[(storeStat.total_quantity > 0) & (storeStat.replace_perc >= 0.1)]

,retailer_id,total_quantity,cancellations,replacements,replace_perc,cancell_perc
id,,,,,,
177,15,193849,13624,19991,0.103127,0.070282
183,15,111561,7539,12487,0.11193,0.067577
185,15,78334,4637,7835,0.10002,0.059195
238,15,60595,5326,6901,0.113887,0.087895
239,15,60721,4407,6652,0.10955,0.072578
246,15,18370,1464,1946,0.105934,0.079695
252,15,10044,678,1151,0.114596,0.067503


In [52]:
#Выгрузим данные
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    storeStat[storeStat.total_quantity == 0].to_excel(writer, sheet_name='ZeroStores')
    storeStat[(storeStat.total_quantity > 0) & (storeStat.replace_perc >= 0.1)].to_excel(writer, sheet_name='StoresStat')

In [30]:
#Составим таблицу, показывающую количество отмен и замен по ритейлерам
retailers = storeStat.retailer_id.value_counts()
retailers = pd.DataFrame({'id':retailers.index, 'total_quantity':retailers.values}).convert_dtypes()
retailers.index = retailers.id
retailers.index.name = "id"
del retailers["id"]
retailers["cancellations"] = retailers["total_quantity"]
retailers["replacements"] = retailers["total_quantity"]
for retid in retailers.index:
    retailers.loc[retid, 'total_quantity'] = storeStat[storeStat.retailer_id == retid].total_quantity.sum()
    retailers.loc[retid, 'cancellations'] = storeStat[storeStat.retailer_id == retid].cancellations.sum()
    retailers.loc[retid, 'replacements'] = storeStat[storeStat.retailer_id == retid].replacements.sum()
retailers = retailers.sort_index()
retailers['replace_perc'] = retailers['replacements']/retailers['total_quantity']
retailers['cancell_perc'] = retailers['cancellations']/retailers['total_quantity']
retailers.sort_values(by="replace_perc", ascending=False)

,total_quantity,cancellations,replacements,replace_perc,cancell_perc
id,,,,,
59,0,0,0,NaN,NaN
61,0,0,0,NaN,NaN
15,1706786,112863,144408,0.084608,0.066126
8,1700517,102180,121411,0.071397,0.060088
16,39164,2431,1692,0.043203,0.062072
1,27475050,1074276,898652,0.032708,0.0391


In [53]:
#Выгрузим данные
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    retailers.to_excel(writer, sheet_name='RetailersStats')

In [32]:
#Посмотрим есть ли товары, которые чаще всего заменяют
replacements.item_id.value_counts()

20971520    1
7621667     1
68473904    1
32779311    1
78920749    1
           ..
78061655    1
37307005    1
64582267    1
10052217    1
56274132    1
Name: item_id, Length: 1250244, dtype: int64

In [33]:
#Составим таблицу совмещающую заказы время доставки
delstat = orders[orders.state == "shipped"][['id', 'delivery_window_id', 'created_at','shipped_at',
    'ship_address_id']].merge(delivery_windows, 
    left_on="delivery_window_id", right_on="id", how='inner')
del delstat["store_id"]
del delstat["id_y"]
delstat.index = delstat["id_x"]
delstat.index.name = "id"
del delstat["id_x"]
delstat

,delivery_window_id,created_at,shipped_at,ship_address_id,starts_at,ends_at,time_zone
id,,,,,,,
756927,226459,2018-08-26 02:06:55,2018-08-26 15:01:42,888330,2018-08-26 14:00:00,2018-08-26 16:00:00,Europe/Moscow
756983,226459,2018-08-26 06:12:13,2018-08-26 15:09:11,888420,2018-08-26 14:00:00,2018-08-26 16:00:00,Europe/Moscow
756963,226460,2018-08-26 05:43:20,2018-08-26 16:28:41,888394,2018-08-26 15:00:00,2018-08-26 17:00:00,Europe/Moscow
756985,226462,2018-08-26 06:13:59,2018-08-26 18:57:01,888422,2018-08-26 17:00:00,2018-08-26 19:00:00,Europe/Moscow
757066,226458,2018-08-26 07:40:58,2018-08-26 13:46:48,870688,2018-08-26 13:00:00,2018-08-26 15:00:00,Europe/Moscow
...,...,...,...,...,...,...,...
5402852,973250,2019-12-17 17:19:32,2019-12-18 11:36:41,7004013,2019-12-18 10:00:00,2019-12-18 12:00:00,Asia/Omsk
5417362,973250,2019-12-18 06:55:18,2019-12-18 10:24:47,7113827,2019-12-18 10:00:00,2019-12-18 12:00:00,Asia/Omsk
5414391,973249,2019-12-18 05:14:22,2019-12-18 10:20:22,7017256,2019-12-18 09:00:00,2019-12-18 11:00:00,Asia/Omsk


In [34]:
#Посмотрим наличие заказов, которые приехали до начала доставки
delstat[delstat.shipped_at < delstat.starts_at]

,delivery_window_id,created_at,shipped_at,ship_address_id,starts_at,ends_at,time_zone
id,,,,,,,
762283,228299,2018-08-28 16:47:24,2018-08-28 18:34:07,877447,2018-08-28 19:00:00,2018-08-28 21:00:00,Europe/Moscow
758427,228285,2018-08-26 19:40:18,2018-08-28 05:55:54,810930,2018-08-28 06:00:00,2018-08-28 08:00:00,Europe/Moscow
759228,228289,2018-08-27 10:01:35,2018-08-28 09:37:18,892050,2018-08-28 10:00:00,2018-08-28 12:00:00,Europe/Moscow
779187,241548,2018-09-07 08:32:26,2018-09-13 08:11:57,926006,2018-09-13 16:00:00,2018-09-13 18:00:00,Europe/Moscow
759937,232431,2018-08-27 14:52:54,2018-09-02 14:39:58,893546,2018-09-02 15:00:00,2018-09-02 17:00:00,Europe/Moscow
...,...,...,...,...,...,...,...
5351014,969476,2019-12-16 11:09:36,2019-12-17 04:33:22,7029754,2019-12-17 05:00:00,2019-12-17 07:00:00,Asia/Omsk
5353295,969476,2019-12-16 11:59:26,2019-12-17 04:14:35,6767873,2019-12-17 05:00:00,2019-12-17 07:00:00,Asia/Omsk
5383817,969485,2019-12-17 09:44:15,2019-12-17 13:42:56,7073061,2019-12-17 14:00:00,2019-12-17 16:00:00,Asia/Omsk


In [54]:
#Выгрузим данные
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    delstat[delstat.shipped_at < delstat.starts_at].to_excel(writer, sheet_name='ArrBefStart')

In [36]:
#Отредактируем таблицу
Ndelstat = delstat[delstat.shipped_at > delstat.starts_at]
Ndelstat["arr_time"] = delstat["shipped_at"]-delstat["starts_at"]
Ndelstat.describe()

<ipython-input-36-b39871afe853>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ndelstat["arr_time"] = delstat["shipped_at"]-delstat["starts_at"]


,delivery_window_id,ship_address_id,arr_time
count,666735.000000,6.667350e+05,666735
mean,694245.764800,3.499564e+06,0 days 01:42:16.972830284
std,213552.713064,1.822658e+06,0 days 05:41:35.007921228
min,226458.000000,6.017000e+03,0 days 00:00:01
25%,530319.000000,1.893788e+06,0 days 00:46:00
50%,742317.000000,3.239593e+06,0 days 01:18:25
75%,880666.000000,5.136772e+06,0 days 01:57:00
max,996054.000000,7.146402e+06,66 days 19:20:36


In [37]:
#Посмотри заказы, которые достовлялись более 2 дней
Ndelstat[(Ndelstat.arr_time >= pd.to_timedelta('2 days')) & (Ndelstat.shipped_at > Ndelstat.ends_at)]

,delivery_window_id,created_at,shipped_at,ship_address_id,starts_at,ends_at,time_zone,arr_time
id,,,,,,,,
886723,285380,2018-11-01 08:06:36,2018-11-05 06:33:31,977008,2018-11-02 05:00:00,2018-11-02 09:00:00,Europe/Moscow,3 days 01:33:31
845923,265063,2018-10-11 13:11:40,2018-10-14 19:05:18,574193,2018-10-11 19:00:00,2018-10-11 21:00:00,Europe/Moscow,3 days 00:05:18
866172,285395,2018-10-22 09:44:26,2018-11-05 06:33:35,1066387,2018-11-02 16:00:00,2018-11-02 18:00:00,Europe/Moscow,2 days 14:33:35
884018,285382,2018-10-30 20:54:05,2018-11-05 06:33:38,1109060,2018-11-02 06:00:00,2018-11-02 14:00:00,Europe/Moscow,3 days 00:33:38
935122,310134,2018-11-20 16:41:09,2018-12-03 15:19:01,1176516,2018-11-23 12:00:00,2018-11-23 14:00:00,Europe/Moscow,10 days 03:19:01
...,...,...,...,...,...,...,...,...
1455454,504896,2019-05-14 06:12:46,2019-05-24 12:09:06,1940199,2019-05-14 12:00:00,2019-05-14 14:00:00,Europe/Samara,10 days 00:09:06
1208869,435634,2019-03-16 04:51:20,2019-03-19 16:34:18,1628590,2019-03-17 04:00:00,2019-03-17 06:00:00,Asia/Omsk,2 days 12:34:18
1428840,497292,2019-05-08 06:58:25,2019-05-24 12:09:09,1848253,2019-05-08 11:00:00,2019-05-08 13:00:00,Asia/Omsk,16 days 01:09:09


In [55]:
#Выгрузим данные
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a') as writer:
    Ndelstat[(Ndelstat.arr_time >= pd.to_timedelta('2 days')) & (Ndelstat.shipped_at > Ndelstat.ends_at)].to_excel(writer, 
        sheet_name='ArrAft2days')

In [46]:
#Посмотрим сколько заказов доставляют полсе конца доставки
print(Ndelstat[Ndelstat.shipped_at > Ndelstat.ends_at].count())
Ndelstat.count()

delivery_window_id    132952
created_at            132952
shipped_at            132952
ship_address_id       132952
starts_at             132952
ends_at               132952
time_zone             132952
arr_time              132952
dtype: int64


delivery_window_id    666735
created_at            666735
shipped_at            666735
ship_address_id       666735
starts_at             666735
ends_at               666735
time_zone             666735
arr_time              666735
dtype: int64

In [39]:
#Составим таблицу, включающую адреса доставки и time_zone
adrstat = delstat.drop(['created_at', 'shipped_at', 'starts_at', 'ends_at'], axis=1)
adrstat = adrstat.merge(addresses, left_on="ship_address_id", right_on="id", how="inner")
adrstat = adrstat.drop(['id', 'elevator', 'floor'], axis=1)
adrstat

,delivery_window_id,ship_address_id,time_zone,lat,lon
0,226459,888330,Europe/Moscow,55.855683,37.644165
1,226459,888420,Europe/Moscow,55.890288,37.612822
2,226460,888394,Europe/Moscow,55.858791,37.638656
3,226462,888422,Europe/Moscow,55.878475,37.620050
4,226458,870688,Europe/Moscow,55.825067,37.621798
...,...,...,...,...,...
729908,973250,7004013,Asia/Omsk,54.983592,73.314520
729909,973250,7113827,Asia/Omsk,54.934267,73.407208
729910,973249,7017256,Asia/Omsk,54.984501,73.220430
729911,973249,7111684,Asia/Omsk,55.024899,73.370359


In [40]:
#Посмотрим сколько заказывают люди в различных регионах
print(adrstat.time_zone.value_counts().sum())
adrstat.time_zone.value_counts()

729913


Europe/Moscow         631497
Europe/Samara          47745
Asia/Yekaterinburg     29408
Asia/Omsk              21148
Europe/Kaliningrad       115
Name: time_zone, dtype: int64

In [56]:
#Выгрузим данные для москвы для дальнейшего анализа в другой среде
adrstat[adrstat.time_zone == "Europe/Moscow"].to_csv('out.csv', encoding='utf-8', index=False)